In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

Mounted at /content/drive


**DEFINICION DE VARIABLES PARA LA EXTRACCION**

In [ ]:
#Número de la semana del mes
semana = 2
#Día en el que empieza la semana (domingo)
inicio_semana = 10
#Día en el que termina la semana (sábado)
fin_semana = 16
#Número del mes
num_mes = 9
#Mes de la extracción
mes = 'septiembre'
#Año de la extracción
anio = 2023

url = 'https://mexico.as.com/tikitakas/horoscopos-del-{}-al-{}-de-{}-{}-prediccion-segun-mhoni-vidente-para-los-12-signos-n/'.format(inicio_semana,fin_semana,mes,anio)

'https://mexico.as.com/tikitakas/horoscopos-del-10-al-16-de-septiembre-2023-prediccion-segun-mhoni-vidente-para-los-12-signos-n/'

**SE CARGA EL ARCHIVO CON EL HISTORICO DE DATOS**

In [ ]:
data = pd.read_excel('horoscopos.xlsx')

**EXTRACCION DEL CONTENIDO DE LA PAGINA**

In [ ]:
r = requests.get(url)
soup = BeautifulSoup(r.content)
span_h = soup.find_all('h3',class_ = '')
span_desc =soup.find_all('b',class_ = '')

if len(span_desc) == 0:
  print('''
          NO EXISTE LA PAGINA
  ''')

**OBTENCION DE LOS SIGNOS**

In [ ]:
span_h = list(span_h)
signos = [str(x) for x in span_h]
signos = [x[re.search('>',x).start()+1:] for x in signos]
signos = [x[:re.search('<',x).start()] for x in signos]
signos = [x.strip() for x in signos]

#signos = signos[0:5] + ['Virgo'] + signos[5:]

if len(signos) != 12:
  print('''
          FALTAN SIGNOS
  ''')
  display(signos)
else:
  print('OK')
  display(signos)

OK


['Aries',
 'Tauro',
 'Géminis',
 'Cáncer',
 'Leo',
 'Virgo',
 'Libra',
 'Escorpión',
 'Sagitario',
 'Capricornio',
 'Acuario',
 'Piscis']

**EXTRACCION DEL HOROSCOPO**

In [ ]:
span_desc_2 = soup.find_all('p', class_='')
span_desc_2 = list(span_desc_2)
span_desc_2 = [str(x).replace('<b>','').replace('</b>','').replace('</p>','').replace('<p class="">','') for x in span_desc_2]

span_desc_2 = span_desc_2[1:-2]
######
i=0
while i>-1:
  try:
    i = [re.compile('<').match(x) is None for x in span_desc_2].index(False)
    span_desc_2.pop(i)
  except:
    i -= 1
######

#span_desc_2
if len(span_desc_2) > 12:
  print('''

        CAMBIÓ LA ESTRUCTURA DEL CONTENIDO

  ''')
  display(span_desc_2)
else:
  print('OK')

OK


**SE AGREGAN COLUMNAS DE IDENTIFICACION**

In [ ]:
base = pd.DataFrame(list(zip(signos, span_desc_2)), columns = ['Signo','Horoscopo'])
base['Ini_semana'] = inicio_semana
base['Fin_semana'] = fin_semana
base['Mes'] = mes
base['Anio'] = anio
base['num_mes'] = num_mes
base['num_semana'] = semana
base

,Signo,Horoscopo,Ini_semana,Fin_semana,Mes,Anio,num_mes,num_semana
0,Aries,La carta de la ‘Rueda de la Fortuna’ nos dice ...,10,16,septiembre,2023,9,2
1,Tauro,La carta de ‘El Diablo’ nos dice que te tienes...,10,16,septiembre,2023,9,2
2,Géminis,"Te sale el ‘AS de Oros’, lo que significa que ...",10,16,septiembre,2023,9,2
3,Cáncer,"Es tu época de brillar, estás cumpliendo años ...",10,16,septiembre,2023,9,2
4,Leo,La carta de ‘El Loco’ nos dice que tienes que ...,10,16,septiembre,2023,9,2
5,Virgo,La carta de ‘El Sol’ nos dice que es el tiempo...,10,16,septiembre,2023,9,2
6,Libra,"La carta de ‘El Mago’ nos dice que pidas, pues...",10,16,septiembre,2023,9,2
7,Escorpión,La carta de ‘El Emperador’ nos dice que estás ...,10,16,septiembre,2023,9,2
8,Sagitario,La carta de ‘El Mundo’ nos dice que será una s...,10,16,septiembre,2023,9,2
9,Capricornio,La carta de ‘AS de Espadas’ nos dice que tendr...,10,16,septiembre,2023,9,2


**Extracción de los número mágicos**

In [ ]:
#Se extraen las clases que contienen los numeros, se convierten a lista y sus elementos a texto
num_dias_color = list(soup.find_all('span', class_=''))
num_dias_color = [str(x) for x in num_dias_color]
p = re.compile('[Nn][úu]mero. m[áa]gico.')
numeros = []
for x in num_dias_color:
  t_f = p.findall(x)
  numeros.append(t_f)

#Se identifica el indide de las lineas de interes
indice = []
for x in range(len(numeros)):
  i = len(numeros[x])
  if i >0:
    indice.append(x)

#con la ayuda de los indices y Regex se extraen solo los numeros
num_mag = []
p = re.compile(r'\d+')
for n in indice:
  txt = num_dias_color[n]
  _ = p.findall(txt)
  num_mag.append(_)
#Se elimina el último en caso de recibir 3 números
for l in num_mag:
  if len(l) > 2:
    l.pop(2)

#Conversion de la lista a dataframe y a formato int
num_mag = pd.DataFrame(num_mag, columns = ['num_1','num_2'])
num_mag['num_1'].apply(lambda x: int(x))
num_mag['num_2'].apply(lambda x: int(x))
num_mag

#Se agregan los numeros a la base

base = pd.concat([base,num_mag], axis=1)
base

,Signo,Horoscopo,Ini_semana,Fin_semana,Mes,Anio,num_mes,num_semana,num_1,num_2
0,Aries,La carta de la ‘Rueda de la Fortuna’ nos dice ...,10,16,septiembre,2023,9,2,7,18
1,Tauro,La carta de ‘El Diablo’ nos dice que te tienes...,10,16,septiembre,2023,9,2,9,27
2,Géminis,"Te sale el ‘AS de Oros’, lo que significa que ...",10,16,septiembre,2023,9,2,2,6
3,Cáncer,"Es tu época de brillar, estás cumpliendo años ...",10,16,septiembre,2023,9,2,10,33
4,Leo,La carta de ‘El Loco’ nos dice que tienes que ...,10,16,septiembre,2023,9,2,5,20
5,Virgo,La carta de ‘El Sol’ nos dice que es el tiempo...,10,16,septiembre,2023,9,2,22,31
6,Libra,"La carta de ‘El Mago’ nos dice que pidas, pues...",10,16,septiembre,2023,9,2,3,30
7,Escorpión,La carta de ‘El Emperador’ nos dice que estás ...,10,16,septiembre,2023,9,2,11,27
8,Sagitario,La carta de ‘El Mundo’ nos dice que será una s...,10,16,septiembre,2023,9,2,15,26
9,Capricornio,La carta de ‘AS de Espadas’ nos dice que tendr...,10,16,septiembre,2023,9,2,14,26


In [ ]:
signo = 'Aries'
x = base.loc[(base['Signo']==signo) & (base['num_mes']==num_mes) & (base['num_semana']==semana)&(base['Anio']==anio),'Horoscopo']
list(x)

['La carta de la ‘Rueda de la Fortuna’ nos dice que son tiempos de estar para arriba, habrá una recompensa económica, por un trabajo que habías hecho en el pasado, tienes que sanar tu economía, pagar tus deudas y empezar a ahorrar, ver si tienes que salir de viaje para el mes de septiembre. Es el tiempo de poner un negocio o tener un cambio en cuestiones laborales. No te detengas, sigue progresando en tu vida. Quítate los malos pensamientos. Te buscarán dos amores nuevos.']

**SE AGREGA LA NUEVA INFORMACION AL HISTORICO**

In [ ]:
base = pd.concat([data,base]).reset_index(drop=True)

In [ ]:
base.to_excel('/content/drive/MyDrive/Colab Notebooks/Horoscopos/horoscopos.xlsx', index=False)

In [ ]:
base = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Horoscopos/horoscopos.xlsx')

**VALIDACIÓN DE LA INTEGRIDAD DE LOS DATOS**

In [ ]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Signo       264 non-null    object
 1   Horoscopo   264 non-null    object
 2   Ini_semana  264 non-null    int64 
 3   Fin_semana  264 non-null    int64 
 4   Mes         264 non-null    object
 5   Anio        264 non-null    int64 
 6   num_mes     264 non-null    int64 
 7   num_semana  264 non-null    int64 
 8   num_1       264 non-null    int64 
 9   num_2       264 non-null    int64 
dtypes: int64(7), object(3)
memory usage: 20.8+ KB


In [ ]:
base['Signo'].value_counts()

Aries          22
Tauro          22
Géminis        22
Cáncer         22
Leo            22
Virgo          22
Libra          22
Escorpión      22
Sagitario      22
Capricornio    22
Acuario        22
Piscis         22
Name: Signo, dtype: int64